In [1]:
import pandas as pd
import folium as fm
from folium.plugins import MarkerCluster
from random import sample

In [2]:
# The columns in the data are:
    # 'ano' is the accident number
    # 'tway' stands for "Trafficway"
    # `day` is the day of the week
    # 'lat' is the latitude
    # 'lon' is the longitude

# Make an interactive map using Folium

In [3]:
# Load the data
df = pd.read_csv('../usc_data/sc_loc2018.csv')
print(f"Initial length of the data: {df.shape[0]}")

df = df.sample(frac=.5)
print(f"New length of the data: {df.shape[0]}")

Initial length of the data: 142406
New length of the data: 71203


In [4]:
# Convert lat and lon to correct decimal degrees
df['lat'] = df['lat'] / 1_000_000
df['lon'] = - (df['lon'] / 1000000)  # Note the negative sign for longitude

# Remove rows with invalid lat/lon
df = df.dropna(subset=['lat', 'lon'])
print(f"Length of the data after removing rows with invalid lat/lon: {df.shape[0]}")

Length of the data after removing rows with invalid lat/lon: 71203


In [5]:
# Create a map centered on South Carolina
sc_center_lat, sc_center_lon = 33.8361, -81.1637  # Approximate center of SC
m = fm.Map(location=[sc_center_lat, sc_center_lon], zoom_start=7)

# Create a MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

In [6]:
# The `tway` column has this meaning:
# 1. Two-way, not divided
# 2. Two-way, divided, unprotected median
# 3. Two-way, divided, barrier
# 4. One way
# 8. Other
# Create mapping for the `tway` column:
tway_map = {1: 'Two-way, not divided',
            2: 'Two-way, divided, unprotected median',
            3: 'Two-way, divided, barrier',
            4: 'One way',
            8: 'Other'}

# The `day` column has this meaning:
# 1. Sunday
# 2. Monday and so on...
# Create mapping for the `day` column:
day_map = {1: 'Sunday',
           2: 'Monday',
           3: 'Tuesday',
           4: 'Wednesday',
           5: 'Thursday',
           6: 'Friday',
           7: 'Saturday'}

In [7]:
possible_colors = ['blue', 'darkgreen', 'cadetblue', 'lightred', 
                   'beige', 'pink', 'green', 'darkred', 'lightgreen', 
                   'lightblue', 'darkblue', 'darkpurple', 'gray', 
                   'purple', 'orange', 'lightgray', 'red', 'black']  # Not including white

# Define color mapping for 'tway'
color_map = {num: col for num, col in zip(df['tway'].unique(), sample(possible_colors, len(df['tway'].unique())))}
color_map

{3: 'beige', 1: 'lightgreen', 2: 'lightred', 4: 'red', 8: 'black'}

In [8]:
# Add markers to the cluster
for idx, row in df.iterrows():    
    fm.Marker(
        popup = fm.Popup(f"""
        <b>Accident Number:</b> {row['ano']}<br>
        <b>Latitude:</b> {row['lat']}<br>
        <b>Longitude:</b> {row['lon']}<br>
        <b>Trafficway:</b> {tway_map.get(row['tway'], 'Other')}<br>
        <b>Day:</b> {day_map.get(row['day'], 'Unknown')}<br>
        """, max_width="100%"),
        location=[row['lat'], row['lon']],
        icon=fm.Icon(color=color_map.get(row['tway'], 'gray')),
        lazy=True
    ).add_to(marker_cluster)

In [9]:
# Create a list to hold each line of the legend
legend_lines = []

# Iterate over the color_map dictionary
for tway, color in color_map.items():
    # Create a line for the legend
    line = f'&nbsp; <i class="fa fa-map-marker fa-2x" style="color:{color}"></i>&nbsp; {tway_map.get(tway, "Other")} <br>'
    # Add the line to the list
    legend_lines.append(line)

# Join all the lines together to form the complete legend
legend_content = '\n'.join(legend_lines)

# Create the legend HTML
legend_html = f'''
<div style="position: fixed; bottom: 20px; left: 50px; width: auto; height: auto;
    border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
    padding: 10px;">&nbsp; <b> Type of Way </b><br>
    {legend_content}
</div>
'''

# Add the legend to the map
m.get_root().html.add_child(fm.Element(legend_html));
print(legend_html)


<div style="position: fixed; bottom: 20px; left: 50px; width: auto; height: auto;
    border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
    padding: 10px;">&nbsp; <b> Type of Way </b><br>
    &nbsp; <i class="fa fa-map-marker fa-2x" style="color:beige"></i>&nbsp; Two-way, divided, barrier <br>
&nbsp; <i class="fa fa-map-marker fa-2x" style="color:lightgreen"></i>&nbsp; Two-way, not divided <br>
&nbsp; <i class="fa fa-map-marker fa-2x" style="color:lightred"></i>&nbsp; Two-way, divided, unprotected median <br>
&nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i>&nbsp; One way <br>
&nbsp; <i class="fa fa-map-marker fa-2x" style="color:black"></i>&nbsp; Other <br>
</div>



In [10]:
# Add borders to the map for South Carolina
# Source: https://nagasudhir.blogspot.com/2021/07/draw-borders-from-geojson-paths-in.html
# style options - https://leafletjs.com/reference-1.7.1.html#path
bordersStyle = {
    'color': 'green',
    'weight': 2,
    'fillColor': 'blue',
    'fillOpacity': 0.1
}

# File (`south carolina.geojson`) downloaded from https://github.com/glynnbird/usstatesgeojson/blob/master/south%20carolina.geojson
fm.GeoJson(
    data=(open("south carolina.geojson", 'r').read()),
    name="South Carolina",
    style_function=lambda x: bordersStyle).add_to(m);

In [11]:
# Save the map
f_name: str = "sc_incidents_map_50.html"
# f_name: str = "sc_incidents_map.html"
m.save(f_name)
print(f"Map has been saved as {f_name}")

Map has been saved as sc_incidents_map_50.html
